In [ ]:
!wget -q -nc http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

In [ ]:
import tarfile

with tarfile.open('aclImdb_v1.tar.gz','r') as tar:
  tar.extractall()

In [ ]:
pip install pyprind


In [ ]:
import pyprind
import pandas as pd
import os

In [ ]:
basepath = 'aclImdb'

labels = {'pos':1, 'neg':0}


In [ ]:
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
for s in ('test','train'):
  for l in ('pos','neg'):
    path = os.path.join(basepath, s, l)
    for file in sorted(os.listdir(path)):
      with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
        txt = infile.read()
      df = pd.concat([df, pd.DataFrame([txt, labels[l]]).T], ignore_index=True, axis=0)
      pbar.update()

df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:09


In [ ]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index=False, encoding='utf-8')


In [ ]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)


,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


# **Vectorización**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array(['The sun is shining',
                 'The weather is sweet',
                 'The sun is shining, the weather is sweet',
                 'and one and one is two'])

bag = count.fit_transform(docs)

In [ ]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [ ]:
print(bag.toarray())


[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [0 2 0 1 1 1 2 0 1]
 [2 1 2 0 0 0 0 1 0]]


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.         0.37632116 0.         0.56855566 0.56855566 0.
  0.46029481 0.         0.        ]
 [0.         0.37632116 0.         0.         0.         0.56855566
  0.46029481 0.         0.56855566]
 [0.         0.4574528  0.         0.3455657  0.3455657  0.3455657
  0.55953044 0.         0.3455657 ]
 [0.65680405 0.1713738  0.65680405 0.         0.         0.
  0.         0.32840203 0.        ]]


In [ ]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

# **Limpieza**

In [ ]:
import re
def preprocessor(text):
  text = re.sub('<[^>]*>','',text)
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
  text = (re.sub('[\W]+', ' ',text.lower())+ ' '.join(emoticons).replace('-',''))
  return text

In [ ]:
preprocessor(df.loc[0,'review'][-50:])

'is seven title brazil not available'

In [ ]:
df['review'] = df['review'].apply(preprocessor)

## **tokenización**

In [ ]:
def tokenizer(text):
  return text.split()

tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [ ]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
  return [porter.stem(word) for word in text.split()]

tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [ ]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(stop_words='english',
                        max_df=.1,
                        max_features=5000)
X = count.fit_transform(df['review'].values)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10,
                                random_state=123,
                                learning_method='batch')
X_topics = lda.fit_transform(X)

In [ ]:
lda.components_.shape


(10, 5000)

In [ ]:
type(count)


sklearn.feature_extraction.text.CountVectorizer

In [ ]:
n_top_words = 10
feature_names = count.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
  print(f'Topic: {topic_idx + 1}')
  print(' '.join([feature_names[i] for i in topic.argsort()[:-n_top_words-1:-1]]))

Topic: 1
horror original comedy black house version classic genre white night
Topic: 2
worst minutes guy script money boring stupid waste wasn comedy
Topic: 3
book dvd read version watched original video remember tv novel
Topic: 4
family performance father beautiful mother woman role lives true performances
Topic: 5
series episode tv kids comedy episodes shows family fun season
Topic: 6
murder police wife john plays crime woman role thriller town
Topic: 7
documentary camera effects audience sense human use style art shot
Topic: 8
music song songs musical role dance rock performance star dancing
Topic: 9
horror effects guy dead budget gore killer special blood looks
Topic: 10
action war game fight american japanese hero battle animation fighting


In [ ]:
bad_movie = X_topics[:, 1].argsort()[::-1]
for iter_idx, movie_idx in enumerate(bad_movie[:3]):
  print(f'Bad movie {iter_idx+1}')
  print(df['review'][movie_idx][:300],'...')


Bad movie 1
i have to admit when i went to see this movie i didn t really have high expectations but even with my low expectations i was totally and utterly disappointed basically luke wilson is a hot shot who tends to go out with slightly crazy girlfriends there s slight mention of a girl stalking him but that ...
Bad movie 2
larry bishop directs writes and leads this soft core porn plot less biker movie about nothing to do with anything to call this one of the worst movies of 2008 is being kind to the garbage that i spent money on while in theaters its one of the worst movies i have ever seen i felt sorry for the girls  ...
Bad movie 3
okay so i found out about this movie and i watched the preview read almost all the reviews and was having a hard time debating whether i should watch it or not before i even watched the movie i was emotionally weird on it i was so unsure if i was going to watch this and be disturbed for like a long  ...


In [ ]:
X_train = df.loc[:35000, 'review'].values
y_train = df.loc[:35000, 'sentiment'].values
X_test = df.loc[35000:,'review'].values
y_test = df.loc[35000:, 'sentiment'].values

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None,
                        stop_words=None,
                        tokenizer=tokenizer,
                        ngram_range=(1,1))

lr_tfidf = Pipeline([('vect',tfidf),
                     ('clf', LogisticRegression(random_state=1000,
                                                solver='liblinear',
                                                C=100000000000000,
                                                penalty='l1'
                                                ))])

In [ ]:
lr_tfidf.fit(X_train,y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 TfidfVectorizer(lowercase=False,
                                 tokenizer=<function tokenizer at 0x77fe75ddd3f0>)),
                ('clf',
                 LogisticRegression(C=100000000000000, penalty='l1',
                                    random_state=1000, solver='liblinear'))])

In [ ]:
lr_tfidf.score(X_test,y_test)

0.8316

In [ ]:
lr_tfidf.predict(np.array(["This is awfull"]))

array([0])

In [ ]:
type(X_test[0])

str

In [ ]:
y_test[:10]


array([0, 1, 0, 0, 1, 0, 1, 1, 1, 1])

In [ ]:
import pickle

In [ ]:
with open("model.pickle", "wb") as f:
  pickle.dump(lr_tfidf, f)